In [1]:
import numpy as np
import snorkel
from snorkel.labeling import labeling_function
from snorkel.labeling import LabelingFunction
from snorkel.labeling import PandasLFApplier
from snorkel.labeling import LFAnalysis
import re
from snorkel.labeling.model import MajorityLabelVoter
import json
import pandas as pd
import label_improve

In [2]:
%load_ext autoreload
%autoreload 2
import label_improve as li

In [3]:
# Loading the data 
dataset_name = "chemprot"

idx_to_label = json.load(open(f"../weak_datasets/{dataset_name}/label.json"))
label_to_idx = {l:i for i,l in idx_to_label.items()}
valid_df = li.chemprot_to_df(json.load(open(f"../weak_datasets/{dataset_name}/valid.json", "r")))
train_df = li.chemprot_to_df(json.load(open(f"../weak_datasets/{dataset_name}/train.json", "r")))
test_df = li.chemprot_to_df(json.load(open(f"../weak_datasets/{dataset_name}/test.json", "r")))

# Sample a dev set to help seed ideas for LFs
dev_df = train_df.sample(250, random_state=123)

In [12]:
# chemprot functions:

ABSTAIN = -1
### Keyword based labeling functions ###

## Part of
#0
@labeling_function()
def lf_amino_acid(x):
    return 0 if 'amino acid' in x.text.lower() else ABSTAIN
#1
@labeling_function()
def lf_replace(x):
    return 0 if 'replace' in x.text.lower() else ABSTAIN
#2 TODO: 0.1988
@labeling_function()
def lf_mutant(x):
    return 0 if 'mutant' in x.text.lower() or 'mutat' in x.text.lower() else ABSTAIN
#3
## Regulator
@labeling_function()
def lf_bind(x):
    return 1 if 'bind' in x.text.lower() else ABSTAIN
#4
@labeling_function()
def lf_interact(x):
    return 1 if 'interact' in x.text.lower() else ABSTAIN
#5
@labeling_function()
def lf_affinity(x):
    return 1 if 'affinit' in x.text.lower() else ABSTAIN
#6 TODO: 0.3578
## Upregulator
# Activator
@labeling_function()
def lf_activate(x):
    return 2 if 'activat' in x.text.lower() else ABSTAIN
#7
@labeling_function()
def lf_increase(x):
    return 2 if 'increas' in x.text.lower() else ABSTAIN
#8 TODO: 
@labeling_function()
def lf_induce(x):
    return 2 if 'induc' in x.text.lower() else ABSTAIN
#9 TODO: 
@labeling_function()
def lf_stimulate(x):
    return 2 if 'stimulat' in x.text.lower() else ABSTAIN
#10
@labeling_function()
def lf_upregulate(x):
    return 2 if 'upregulat' in x.text.lower() else ABSTAIN
#11
## Downregulator
@labeling_function()
def lf_downregulate(x):
    return 3 if 'downregulat' in x.text.lower() or 'down-regulat' in x.text.lower() else ABSTAIN
#12
@labeling_function()
def lf_reduce(x):
    return 3 if 'reduc' in x.text.lower() else ABSTAIN
#13
@labeling_function()
def lf_inhibit(x):
    return 3 if 'inhibit' in x.text.lower() else ABSTAIN
#14
@labeling_function()
def lf_decrease(x):
    return 3 if 'decreas' in x.text.lower() else ABSTAIN
#15
## Agonist
@labeling_function()
def lf_agonist(x):
    return 4 if ' agoni' in x.text.lower() or "\tagoni" in x.text.lower() else ABSTAIN

#16
## Antagonist
@labeling_function()
def lf_antagonist(x):
    return 5 if 'antagon' in x.text.lower() else ABSTAIN

#17
## Modulator
# TODO: Delete this LF, or change this to modulator ??
@labeling_function()
def lf_modulate(x):
    return 6 if 'modulat' in x.text.lower() else ABSTAIN

#18
@labeling_function()
def lf_allosteric(x):
    return 6 if 'allosteric' in x.text.lower() else ABSTAIN
#19
## Cofactor
@labeling_function()
def lf_cofactor(x):
    return 7 if 'cofactor' in x.text.lower() else ABSTAIN
#20
## Substrate/Product
@labeling_function()
def lf_substrate(x):
    return 8 if 'substrate' in x.text.lower() else ABSTAIN
#21
@labeling_function()
def lf_transport(x):
    return 8 if 'transport' in x.text.lower() else ABSTAIN
#22
@labeling_function()
def lf_catalyze(x):
    return 8 if 'catalyz' in x.text.lower() or 'catalys' in x.text.lower() else ABSTAIN
#23
@labeling_function()
def lf_product(x):
    return 8 if "produc" in x.text.lower() else ABSTAIN
#24
@labeling_function()
def lf_convert(x):
    return 8 if "conver" in x.text.lower() else ABSTAIN
#25
## NOT
@labeling_function()
def lf_not(x):
    return 9 if 'not' in x.text.lower() else ABSTAIN

# 26 replace the 17 (18)
@labeling_function()
def lf_combined_modulator(x):
    sentence_lower = x.text.lower()

    specific_terms = ['allosteric modulator', 'positive modulator', 'negative modulator', 'non-competitive modulator', 'positive allosteric modulator']
    if any(term in sentence_lower for term in specific_terms):
        return 6

    modulating_terms = ['modulat', 'allosteric', 'potentiate', 'regulate']
    for term in modulating_terms:
        if term in sentence_lower:
            term_index = sentence_lower.index(term)
            if x.entity1_index == -1 or x.entity2_index == -1:
                return ABSTAIN
            if abs(term_index - x.entity1_index) < 20 or abs(term_index - x.entity2_index) < 20:
                return 6
    # check the first indcidences of modulator and positive
    if x.entity1 in sentence_lower and x.entity2 in sentence_lower:
        entity1_index = sentence_lower.index(x.entity1.lower())
        entity2_index = sentence_lower.index(x.entity2.lower())
        between_entities = sentence_lower[min(entity1_index, entity2_index):max(entity1_index, entity2_index)]
        if 'modulate' in between_entities or 'regulate' in between_entities:
            return 6

    if 'positive' in sentence_lower and 'modulator' in sentence_lower:
        pos_indices = [i for i, word in enumerate(sentence_lower.split()) if word == 'positive']
        mod_indices = [i for i, word in enumerate(sentence_lower.split()) if 'modulator' in word]
        min_distance = min(abs(p - m) for p in pos_indices for m in mod_indices)
        if min_distance <= 3:
            return 6

    return ABSTAIN

lfs = [lf_amino_acid, lf_replace, lf_mutant, lf_bind, lf_interact, lf_affinity, lf_activate, lf_increase, lf_induce, lf_stimulate, lf_upregulate, lf_downregulate, lf_reduce, lf_inhibit, lf_decrease, lf_agonist, lf_antagonist, lf_combined_modulator, lf_allosteric, lf_cofactor, lf_substrate, lf_transport, lf_catalyze, lf_product, lf_convert, lf_not]

In [13]:
# show the first row of the dataframe
train_dev = train_df.head(1000)
train_dev = li.chemprot_enhanced(train_dev)
L_dev = li.apply_LFs(lfs, train_dev)
L_dev
print(train_dev.iloc[352])
L_dev[352]

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:00<00:00, 7629.43it/s]

text             While SAR within the HTS series was very shall...
labels                                                           6
entity1                                                         M5
entity2                                                      ML172
span1                                                   [173, 175]
span2                                                   [129, 134]
weak_labels      [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...
entity1_index                                                   28
entity2_index                                                   21
Name: 12446, dtype: object


array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1])

In [14]:
print("Test Coverage:", li.calc_coverage(L_dev))
lf_analysis = LFAnalysis(L_dev, lfs).lf_summary()
# Calculates how many of an LFs votes result in conflicts (helpful signal for debugging LFs)
lf_analysis['Conflict Ratio'] = lf_analysis['Conflicts'] / lf_analysis['Coverage']
lf_analysis

Test Coverage: 0.874


,j,Polarity,Coverage,Overlaps,Conflicts,Conflict Ratio
lf_amino_acid,0,[0],0.015,0.010,0.009,0.600000
lf_replace,1,[0],0.003,0.002,0.002,0.666667
lf_mutant,2,[0],0.029,0.019,0.018,0.620690
lf_bind,3,[1],0.099,0.073,0.064,0.646465
lf_interact,4,[1],0.019,0.014,0.011,0.578947
lf_affinity,5,[1],0.043,0.032,0.026,0.604651
lf_activate,6,[2],0.123,0.093,0.075,0.609756
lf_increase,7,[2],0.111,0.074,0.063,0.567568
lf_induce,8,[2],0.158,0.123,0.102,0.645570
lf_stimulate,9,[2],0.043,0.036,0.030,0.697674


In [15]:
# Calculate accuracy on the validation set (Ideally do this only at the end)
majority_model = MajorityLabelVoter(10)
preds_valid = majority_model.predict(L=L_dev)
print(preds_valid)
print(preds_valid[352])
print((preds_valid[preds_valid != -1] == train_dev[preds_valid != -1].labels.values).mean())
print(train_dev.iloc[352])
incorrect_indices = np.where(preds_valid[preds_valid != -1] != train_dev[preds_valid != -1].labels.values)[0]
incorrect_predictions_df = train_dev.iloc[incorrect_indices]
label_six_df = incorrect_predictions_df[incorrect_predictions_df.labels == 6]
label_six_df

[ 3  8 -1  8  3  1  8  2 -1  2  1 -1  3  3 -1  2  3  1  3 -1  3  1  1  3
  1 -1  8 -1 -1  1 -1 -1  3  2  1 -1 -1 -1 -1  2  3  8 -1  5 -1  3  2  2
 -1  2 -1  3  3 -1  3  3  2 -1 -1  3  5 -1 -1  2  8 -1  3  2  3 -1 -1  3
  2  3  2 -1 -1 -1  3  1  2  1  3 -1 -1 -1  3  3 -1 -1  2  8  3  3  3  1
 -1  8  2 -1  3 -1  1 -1 -1  3 -1 -1 -1 -1  0  3  3 -1 -1 -1  8 -1 -1 -1
  3 -1 -1  1 -1  3 -1  3  2 -1  3  2 -1 -1  3  2  3 -1 -1  1  5  1  1 -1
 -1  3 -1  1  3  1  2  5  3 -1  5 -1 -1 -1 -1 -1 -1  2  9  3  8  4  3  3
 -1  3 -1 -1 -1 -1  9 -1 -1  2  2 -1  3  8  3  2 -1  8  3  2 -1  2  2 -1
 -1  8 -1  2 -1 -1  1 -1 -1 -1  3  2 -1  2  8 -1  3 -1 -1 -1 -1 -1  2 -1
  2  5 -1  3  2 -1 -1  5  2 -1  2  1  2  2  2 -1  5  4  2 -1 -1 -1 -1 -1
  1 -1 -1  3  2  0 -1  3 -1  3 -1 -1  3  8 -1  8  2 -1 -1  5 -1 -1 -1 -1
  2 -1  3 -1  3  3 -1  3 -1  4 -1  8 -1 -1 -1 -1  3  1 -1 -1 -1  2 -1  3
 -1  1  3 -1 -1 -1  3 -1 -1  3 -1  8  8 -1  3  3 -1 -1 -1  1  1  2  3 -1
 -1  3  1  3  3 -1  3 -1  2  3  2 -1  8  5 -1  3 -1

,text,labels,entity1,entity2,span1,span2,weak_labels,entity1_index,entity2_index
12446,While SAR within the HTS series was very shall...,6,M5,ML172,"[173, 175]","[129, 134]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...",28.0,21.0


In [18]:
lfs = [lf_amino_acid, lf_replace, lf_mutant, lf_bind, lf_interact, lf_affinity, lf_activate, lf_increase, lf_induce, lf_stimulate, lf_upregulate, lf_downregulate, lf_reduce, lf_inhibit, lf_decrease, lf_agonist, lf_antagonist, lf_modulate, lf_allosteric, lf_cofactor, lf_substrate, lf_transport, lf_catalyze, lf_product, lf_convert, lf_not]

# show the first row of the dataframe
train_dev = train_df.head(1000)
train_dev = li.chemprot_enhanced(train_dev)
L_dev2 = li.apply_LFs(lfs, train_dev)
L_dev2  


  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:00<00:00, 8209.08it/s]


array([[-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       ...,
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1]])

In [22]:
difference = np.where(L_dev2 !=  L_dev)[0]

In [25]:
count = 0
for i in difference:
    print(train_dev.iloc[i])
    print(train_dev.iloc[i].text)
    print(L_dev[i])
    print(L_dev2[i])
    count +=1
    print("\n")
print(count)

text             This effect appeared to be due to both competi...
labels                                                           1
entity1                                                        Trx
entity2                                          S-nitrosocysteine
span1                                                     [94, 97]
span2                                                     [59, 76]
weak_labels      [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...
entity1_index                                                   15
entity2_index                                                   10
Name: 15523, dtype: object
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  3 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1]
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  3 -1 -1 -1 -1  6 -1 -1 -1 -1 -1 -1
 -1 -1]


text             Thalidomide--removed from widespread clinical ...
labels                                                           3
entity1                                                  TNF alpha
entit

In [ ]:
# Calculate accuracy on the validation set (Ideally do this only at the end)
majority_model = MajorityLabelVoter(10)
preds_valid = majority_model.predict(L=L_dev)
print(preds_valid)
print(preds_valid[352])
print((preds_valid[preds_valid != -1] == train_dev[preds_valid != -1].labels.values).mean())
print(train_dev.iloc[352])
incorrect_indices = np.where(preds_valid[preds_valid != -1] != train_dev[preds_valid != -1].labels.values)[0]
incorrect_predictions_df = train_dev.iloc[incorrect_indices]
label_six_df = incorrect_predictions_df[incorrect_predictions_df.labels == 6]
label_six_df

# DO NOT GO BEYOND THIS LINE

# ------------------------------------------------------------------------------------ #

In [71]:
df = li.chemprot_enhanced(train_dev)
df

,text,labels,entity1,entity2,span1,span2,weak_labels,entity1_index,entity2_index
10305,"Selective inhibition of PDE5 is a rational therapeutic approach in ED, as proved by the clinical success of sildenafil.",3,PDE5,sildenafil,"[24, 28]","[108, 118]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 3, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]",3.0,18.0
8422,"Furthermore, knockdown of OPN enhanced cell death caused by other drugs, including paclitaxel, doxorubicin, actinomycin-D, and rapamycin, which are also P-gp substrates.",8,P-gp,paclitaxel,"[153, 157]","[83, 93]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 8, -1, -1, -1, -1, -1]",20.0,12.0
8486,"Furthermore, no impact on cytokine release (i.e., on IL-10, IL-6, IL-12/23p40 and TNFα levels) was seen in LPS-stimulated human PBMCs, except with JWH-210 and JWH-122 which caused a decrease of TNFα and IL-12/23p40.",3,IL-12,JWH-210,"[203, 208]","[147, 154]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, 2, -1, -1, -1, -1, 3, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]",32.0,22.0
6254,"Among neuroleptics, the four most potent compounds at the human serotonin transporter were triflupromazine, fluperlapine, chlorpromazine, and ziprasidone (K(D) 24-39 nM); and at the norepinephrine transporter, chlorpromazine, zotepine, chlorprothixene, and promazine (K(D) 19-25 nM).",1,norepinephrine transporter,chlorpromazine,"[182, 208]","[210, 224]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 8, -1, -1, -1, -1]",24.5,26.0
1801,"Epidermal growth factor receptor inhibitors currently under investigation include the small molecules gefitinib (Iressa, ZD1839) and erlotinib (Tarceva, OSI-774), as well as monoclonal antibodies such as cetuximab (IMC-225, Erbitux).",3,Epidermal growth factor receptor,Tarceva,"[0, 32]","[144, 151]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 3, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]",1.5,17.0
...,...,...,...,...,...,...,...,...,...
9350,"Tamsulosin, the first prostate-selective alpha 1A-adrenoceptor antagonist.",5,alpha 1A-adrenoceptor,Tamsulosin,"[41, 62]","[0, 10]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 5, -1, -1, -1, -1, -1, -1, -1, -1, -1]",4.5,0.0
13234,"The GRIP1 reduction was inhibited by MK-801, an N-methyl-d-aspartate (NMDA) receptor antagonist, but not by 6-cyano-7-nitroquinoxaline-2,3-dione (CNQX), an AMPA receptor antagonist.",2,GRIP1,MK-801,"[4, 9]","[37, 43]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 3, 3, -1, -1, 5, -1, -1, -1, -1, -1, -1, -1, -1, 9]",1.0,6.0
1837,The reciprocal inhibition of SR-BI and ABCA1 by BLT-4 and glyburide raises the possibility that these proteins may share similar or common steps in their mechanisms of lipid transport.,3,ABCA1,glyburide,"[39, 44]","[58, 67]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 3, -1, -1, -1, -1, -1, -1, -1, 8, -1, -1, -1, -1]",6.0,10.0
11290,"Firstly, transgenic plants overexpressing formate dehydrogenase (FDH, EC 1.2.1.2) were used to continue our previous studies on the function of FDH in formate metabolism.",8,EC 1.2.1.2,formate,"[70, 80]","[151, 158]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]",7.5,22.0


In [72]:
lfs = [lf_combined_modulator]

# Initialize the applier
applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df)
L_train

100%|██████████| 1000/1000 [00:00<00:00, 66886.27it/s]


array([[-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],

In [73]:
# Cell to inspect the dev set for a given label 
label = 'Modulator' # Change this variable
pd.set_option('display.max_colwidth', 1000)
label6 = train_df[train_df['labels'] == int(label_to_idx[label])]
# save the dev set to a file
df2 = li.chemprot_enhanced(label6)
df2

,text,labels,entity1,entity2,span1,span2,weak_labels,entity1_index,entity2_index
12446,"While SAR within the HTS series was very shallow and unable to be optimized, grafting the phenethyl ether linkage onto the ML129/ML172 cores led to the first sub-micromolar M5 PAM, ML326 (VU0467903), (human and rat M5 EC50s of 409nM and 500nM, respectively) with excellent mAChR selectivity (M1-M4 EC50s >30μM) and a robust 20-fold leftward shift of the ACh CRC.",6,M5,ML172,"[173, 175]","[129, 134]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]",28.0,21.0
3477,"Anxiolytic- but not antidepressant-like activity of Lu AF21934, a novel, selective positive allosteric modulator of the mGlu₄ receptor.",6,mGlu₄,Lu AF21934,"[120, 125]","[52, 62]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 6, 6, -1, -1, -1, -1, -1, -1, 9]",16.0,6.5
8655,"Conversely, ovarian PRA and PRB were positively regulated by ethanol and ethanol-melatonin combination, whereas PRA was down-regulated in the uterus and oviduct after ethanol consumption.",6,PRB,melatonin,"[28, 31]","[81, 90]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 3, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 8, -1]",4.0,11.0
3387,"BDZs and other positive GABA(A)R modulators, including barbiturates, ethanol, and neurosteroids, can also inhibit L-type voltage-gated calcium channels (L-VGCCs), which could contribute to reduced neuronal excitability.",6,GABA(A)R,BDZs,"[24, 32]","[0, 4]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 3, 3, -1, -1, -1, 6, -1, -1, -1, -1, -1, -1, -1, -1]",4.0,0.0
8654,"Conversely, ovarian PRA and PRB were positively regulated by ethanol and ethanol-melatonin combination, whereas PRA was down-regulated in the uterus and oviduct after ethanol consumption.",6,PRA,melatonin,"[20, 23]","[81, 90]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 3, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 8, -1]",2.0,11.0
13836,The recently discovered hyperinsulinism/hyperammonemia disorder showed that the loss of allosteric inhibition of GDH by GTP causes excessive secretion of insulin.,6,GDH,GTP,"[113, 116]","[120, 123]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 3, -1, -1, -1, -1, 6, -1, -1, -1, -1, -1, -1, -1]",13.0,15.0
5378,The implications of our finding for mechanism of in vivo actions of rapamycin and for design of novel allosteric drugs targeting the proteasome are discussed.,6,proteasome,rapamycin,"[133, 143]","[68, 77]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 6, -1, -1, -1, -1, -1, -1, -1]",22.0,12.0
3606,Ifenprodil is an allosteric inhibitor of GluN1/GluN2B N-methyl-D-aspartate receptors.,6,GluN1,Ifenprodil,"[41, 46]","[0, 10]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 3, -1, -1, -1, -1, 6, -1, -1, -1, -1, -1, -1, -1]",6.0,0.0
15281,Synthesis and structure-activity relationships of indazole arylsulfonamides as allosteric CC-chemokine receptor 4 (CCR4) antagonists.,6,CC-chemokine receptor 4,indazole arylsulfonamides,"[90, 113]","[50, 75]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 5, -1, 6, -1, -1, -1, -1, -1, -1, -1]",10.0,5.5
3381,Benzodiazepines (BDZs) depress neuronal excitability via positive allosteric modulation of inhibitory GABA(A) receptors (GABA(A)R).,6,GABA(A)R,BDZs,"[121, 129]","[17, 21]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 3, -1, -1, -1, 6, 6, -1, -1, -1, -1, -1, -1, -1]",13.0,1.0


In [9]:
count = 0
for i in range(len(df2)):
    weak_label = df2.iloc[i]['weak_labels'][17]
    if weak_label != -1:
        print(i)
        count+=1
print('count', count)

1
3
9
29
33
36
38
40
41
49
53
count 11


In [10]:
L_train = applier.apply(df=df2)
L_train

100%|██████████| 55/55 [00:00<00:00, 22625.22it/s]


array([[-1],
       [ 6],
       [-1],
       [ 6],
       [-1],
       [-1],
       [-1],
       [ 6],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [ 6],
       [-1],
       [-1],
       [-1],
       [-1],
       [ 6],
       [ 6],
       [-1],
       [-1],
       [-1],
       [-1],
       [ 6],
       [-1],
       [ 6],
       [-1],
       [-1],
       [-1],
       [ 6],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [ 6],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [ 6],
       [ 6],
       [-1],
       [-1],
       [ 6],
       [-1]])

In [11]:


# Cell to inspect the dev set for a given label 
label = 'Modulator' # Change this variable
pd.set_option('display.max_colwidth', 1000)


def get_18th_element(lst):
    return lst[17] if len(lst) > 17  else None

# Create a copy of the DataFrame
train_df_copy = train_df.copy()

# Apply the function to the copy
train_df_copy['18th_weak_label'] = train_df_copy['weak_labels'].apply(get_18th_element)

# Filter the copy
label6_all = train_df[train_df_copy['18th_weak_label'] == 6]

# Continue with your code
df3 = li.chemprot_enhanced(label6_all)
df3

,text,labels,entity1,entity2,span1,span2,weak_labels,entity1_index,entity2_index
15523,This effect appeared to be due to both competition between S-nitrosocysteine and Prx1 for the Trx system and direct modulation by S-nitrosocysteine of Trx reductase activity.,1,Trx,S-nitrosocysteine,"[94, 97]","[59, 76]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 3, -1, -1, -1, -1, 6, -1, -1, -1, -1, -1, -1, -1, -1]",15.0,10.0
6532,"Thalidomide--removed from widespread clinical use by 1962 because of severe teratogenicity--has anti-angiogenic and immunomodulatory effects, including the inhibition of TNF alpha.",3,TNF alpha,Thalidomide,"[170, 179]","[0, 11]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 3, -1, -1, -1, 6, -1, -1, -1, -1, -1, -1, -1, -1]",19.5,0.0
5676,Data suggest that bisphosphonates via modulation of the activity of small-GTPases induce apoptosis in neoplastic cells by DNA-CpG-demethylation and stimulation of FAS-expression.,2,FAS,bisphosphonates,"[163, 166]","[18, 33]","[-1, -1, -1, -1, -1, -1, -1, -1, 2, 2, -1, -1, -1, -1, -1, -1, -1, 6, -1, -1, -1, -1, -1, -1, -1, -1]",21.0,3.0
3367,"BDZs and other positive GABA(A)R modulators, including barbiturates, ethanol, and neurosteroids, can also inhibit L-type voltage-gated calcium channels (L-VGCCs), which could contribute to reduced neuronal excitability.",3,L-VGCCs,neurosteroids,"[153, 160]","[82, 95]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 3, 3, -1, -1, -1, 6, -1, -1, -1, -1, -1, -1, -1, -1]",18.0,10.0
5005,"This study is the first to identify Class V CGPs with their distinctive methine or trimethine linkage between two disubstituted pyrylium moieties as a particularly potent class of MRP modulators and also show that within this core structure, differences in the electronegativity associated with a chalcogen atom can be the sole determinant of whether a compound will stimulate or inhibit MRP2.",1,MRP,methine,"[180, 183]","[72, 79]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, 2, -1, -1, -1, 3, -1, -1, -1, 6, -1, -1, -1, -1, -1, -1, -1, -1]",28.0,13.0
...,...,...,...,...,...,...,...,...,...
5116,"Taken together, our findings indicate that 5HHMF suppresses NO production through modulation of iNOS, consequently suppressing NF-κB activity and induction of Nrf2-dependent HO-1 activity.",3,NF-κB,NO,"[127, 132]","[60, 62]","[-1, -1, -1, -1, -1, -1, -1, -1, 2, -1, -1, -1, -1, -1, -1, -1, -1, 6, -1, -1, -1, -1, -1, 8, -1, -1]",16.0,8.0
491,"There is, however, much information on the direct (acute and chronic) effects of alcohol on the binding properties of opioid receptors, as well as modulation of opioid peptide synthesis and secretion (e.g.",1,opioid peptide,alcohol,"[161, 175]","[81, 88]","[-1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 6, -1, -1, -1, -1, -1, -1, -1, -1]",26.5,13.0
1225,"Raloxifene is a selective ER modulator with less uterine estrogen agonist activity than tamoxifen, and it is hoped that it will result in fewer uterine cancers but will be equally efficacious in reducing the risk of breast cancer.",1,ER,Raloxifene,"[26, 28]","[0, 10]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 3, -1, -1, 4, -1, 6, -1, -1, -1, -1, -1, -1, -1, -1]",4.0,0.0
3384,"BDZs and other positive GABA(A)R modulators, including barbiturates, ethanol, and neurosteroids, can also inhibit L-type voltage-gated calcium channels (L-VGCCs), which could contribute to reduced neuronal excitability.",6,GABA(A)R,barbiturates,"[24, 32]","[55, 67]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 3, 3, -1, -1, -1, 6, -1, -1, -1, -1, -1, -1, -1, -1]",4.0,7.0


In [12]:
df3 = df3.head(30)
count = 0
for i in range(len(df3)):
    label = df3.iloc[i]['labels']
    if label == 6:
        print(i)
        count+=1
print('count', count)

5
11
count 2


In [13]:
L_train = applier.apply(df=df3)
count = 0
for i in range(len(L_train)):
    if L_train[i][0] != -1:
        print(i)
        count+=1
print('count', count)


100%|██████████| 30/30 [00:00<00:00, 23232.85it/s]

2
3
5
7
11
17
20
21
26
29
count 10


## Main Work

In [14]:
df2.to_csv(f"./dev_{label}.csv")
chemprot6 = li.df_to_chemprot(li.chemprot_enhanced(label6))
li.save_dataset(chemprot6, f"./dev_{label}.json")


In [15]:
# Cell to inspect dev set for a given keyword (to inspect conflicts). Note: this doesn't work with "+" 
keyword = 'iPad' # Change this word
pd.set_option('display.max_colwidth', 1000)
dev_df[dev_df['text'].str.contains(keyword)]

,text,labels,entity1,entity2,span1,span2,weak_labels


## Evaluation

In [16]:
L_train = apply_LFs(lfs, train_df)
L_valid = apply_LFs(lfs, valid_df)
L_test = apply_LFs(lfs, test_df)

print("Train Coverage:", calc_coverage(L_train))
print("Valid Coverage:", calc_coverage(L_valid))
print("Test Coverage:", calc_coverage(L_test))

lf_analysis = LFAnalysis(L=L_valid, lfs=lfs).lf_summary()

# Calculates how many of an LFs votes result in conflicts (helpful signal for debugging LFs)
lf_analysis['Conflict Ratio'] = lf_analysis['Conflicts'] / lf_analysis['Coverage']
lf_analysis

NameError: name 'apply_LFs' is not defined

In [ ]:
# List LFs for which 'Conflict Ratio' is above some threshold (helpful for debugging)
lf_analysis[lf_analysis['Conflict Ratio'] > 0.8]['Conflict Ratio'].sort_values(ascending=False)

In [16]:
# Calculate accuracy on the validation set (Ideally do this only at the end)
majority_model = MajorityLabelVoter(10)
preds_valid = majority_model.predict(L)
(preds_valid[preds_valid != -1] == valid_df[preds_valid != -1].labels.values).mean()

NameError: name 'L_valid' is not defined

In [ ]:
json.dump(keywords, open("amazon_LFs_v1.json", "w"), indent=8)

In [ ]:
# Replace the LFs for a given dataset (in wrench format)
# dataset_name = "dbpedia"

# train_json = json.load(open(f"../weak_datasets/{dataset_name}/train.json", "r"))
# for idx in train_json:
#     train_json[idx]['weak_labels'] = [int(i) for i in list(L_train[int(idx)])]
    
# valid_json = json.load(open(f"../weak_datasets/{dataset_name}/valid.json", "r"))
# for idx in valid_json:
#     valid_json[idx]['weak_labels'] = [int(i) for i in list(L_valid[int(idx)])]
    
# test_json = json.load(open(f"../weak_datasets/{dataset_name}/test.json", "r"))
# for idx in test_json:
#     test_json[idx]['weak_labels'] = [int(i) for i in list(L_test[int(idx)])]

# json.dump(train_json, open(f"../weak_datasets/{dataset_name}/train.json", 'w'), indent=4)
# json.dump(valid_json, open(f"../weak_datasets/{dataset_name}/valid.json", 'w'), indent=4)
# json.dump(test_json, open(f"../weak_datasets/{dataset_name}/test.json", 'w'), indent=4)